In [1]:
from dataclasses import dataclass
from typing import List, Dict, Tuple
from enum import Enum

In [13]:
# إضافة الكود الجديد للقيم المرجعية

mcv=(80,100)
rdw=(11,14.5)

@dataclass
class Patient:
    age_months: float
    gender: Gender
    is_pregnant: bool = False
    pregnancy_trimester: Optional[int] = None

# تعديل دوال التصنيف في CBCAnalyzer فقط
class CBCAnalyzer:
    """Main class for comprehensive CBC analysis"""
    
    def __init__(self):
        self.rbc_pattern = RBCPattern()
        self.anemia_classifier = AnemiaClassifier()
        self.wbc_analyzer = WBCAnalyzer()
        self.plt_analyzer = PLTAnalyzer()
        self.reference_values = CBCReferenceValues()
# Severity levels for results

class Severity(Enum):
    NORMAL = "Normal"
    WARNING = "Warning"
    URGENT = "Urgent"
    CRITICAL = "Critical"

# Data classes for CBC components
@dataclass
class CBCResult:
    # RBC related
    hgb: float  # Hemoglobin
    rbc: float  # Red Blood Cell count
    hct: float  # Hematocrit
    mcv: float  # Mean Corpuscular Volume
    rdw: float  # Red Cell Distribution Width
    
    # WBC related
    wbc: float  # White Blood Cell count
    neutrophils: float  # Neutrophil count
    lymphocytes: float  # Lymphocyte count
    
    # PLT related
    plt: float  # Platelet count
    mpv: float  # Mean Platelet Volume

class RBCPattern:
    """Handles RBC pattern analysis and classification"""
    
    @staticmethod
    def classify_pattern(hgb: str, rbc: str, hct: str) -> str:
        """Classifies the HGB-RBC-HCT pattern"""
        pattern = f"{hgb}_{rbc}_{hct}"
        
        # Wrong measurement patterns
        wrong_patterns = [
            "Low_Low_High", "Low_Normal_High", "Low_High_High",
            "Normal_Low_High", "Normal_High_Low", "High_Low_Low",
            "High_Low_High", "High_Low_Normal", "High_Normal_Low",
            "High_High_Low", "High_High_Normal"
        ]
        
        if pattern in wrong_patterns:
            return "Laboratory Error"
            
        # Normal patterns
        normal_patterns = [
            "Normal_Normal_Normal", "Normal_Normal_Low", "Normal_Normal_High",
            "Normal_High_Normal", "Normal_High_High", "High_Normal_High",
            "High_Normal_Normal"
        ]
        
        if pattern == "Normal_Normal_Normal": # check rdw , mcv if not normal  # check distribution
                return "Normal Pattern"
        
        if pattern == "Normal_Normal_Low":
            if mcv < mcv[0]: #early early (Iron Deficiency/Beta Thalassemia Trait)  # check distribution
                return "Early Microcytic Anemia"
            elif mcv > (mcv[1]+5):
                return "Laboratory Error"
            else:
                return "Normal Pattern"
        
        if pattern == "Normal_Normal_High": # check distribution comment :relative low rbc  , relative high mcv Indicates a problem with cell division.
            # if mcv > (mcv[1]+5) and rdw > rdw[1]:
            #     return "Possible Early Macrocytic Anemia and Dehydration"
            #  elif mcv > (mcv[1]+5) and rdw<rdw[1] :
            #     return "adaptation to hypoxemia due to macrocytic anemia"
            # else:
                return "Normal Pattern"

        if pattern == "Normal_High_Normal":
            if mcv < mcv([0]-5) and rdw > rdw[1]:
                return "Early Microcytic Anemia"
            elif mcv < (mcv[0]-10) and rdw<rdw[1] :
                return "Microcytic Anemia"
            elif mcv > mcv[1] :
                return "Laboratory Error"
            else:
                return "Normal Pattern"
        
        if pattern == "Normal_High_High":
            return "Normal Pattern"
        
        if pattern == "High_Normal_High": #check hct 
            return "Normal Pattern"
        
        if pattern == "High_Normal_Normal":
            return "Normal Pattern"

        return "Unknown Pattern"
    
        # Early or not microcytosis patterns [ clearly pattern ( only Microcytosis)]
        early_micro_patterns = ["Low_High_Low", "Low_High_Normal"]
        if pattern in early_micro_patterns:
            if 70 < mcv < mcv[0] and rdw > (rdw[1]-1):
                return "Early Microcytic Anemia"
            elif mcv < (mcv[0]-10) and rdw < rdw[1] and (mcv/rbc)<13 :
                return "Beta Thalassemia"
            elif mcv > mcv[0] :
                return "Laboratory Error"
            else :
                return "Microcytic Anemia"

        # Early macrocytosis patterns [ clearly pattern ( only Macrocytosis)]
        early_macro_patterns = ["Normal_Low_Normal", "Normal_Low_Low"]
        if pattern in early_macro_patterns :
            if mcv > mcv[1] and rdw > rdw[1]:
                return "Possible Early Macrocytic Anemia"
            elif mcv > (mcv[1]) and rdw < rdw[1] :
                return "adaptation to hypoxemia due to macrocytic anemia"
            elif mcv < mcv[0] :
                return "Laboratory Error"
            else:
                return "Normal Pattern but fimmiler to macrocytic "

        # Moderate Anemia [ clearly pattern (Microcytosis  or normo)] :Low_Normal_Normal"(pre low hct) 
        moderate_anemia_patterns = [ "Low_Normal_Normal", "Low_Normal_Low"]
        if pattern  in moderate_anemia_patterns :
            if mcv < mcv[0] :
                return "Microcytic Anemia <13 with min mcv = 10.3"     
            elif mcv[0] >= mcv >=  mcv[1] :
                return "Normocytic anemia but familler to  Microcytic Anemia with min mcv = 10.3"
            elif :#elif mcv > mcv[0] :
                return "Laboratory Error"
            
        # Moderate Anemia [ clearly pattern (Macrocytosis  or normo)] 
        
        if pattern  =="Low_Low_Normal" :
            if mcv > mcv[1] :
                return "[Macrocytic_Anemia]: with min mcv = 11.2"     
            elif mcv[0] >= mcv >=  mcv[1] :
                return "[Normocytic_Anemia]: but familler to  Macrocytic Anemia with min mcv = 10.3"
            else:#elif mcv < mcv[0] :
                return "Laboratory Error"
            
        # Anemia patterns
        if pattern == "Low_Low_Low" :
            # RDW high + MCV low
            if mcv < mcv[0] and rdw > rdw[1] :
                return "[Microcytic_Anemia] except thalassemia"
            # RDW normal + MCV low 
            elif mcv < mcv[0] and rdw < rdw[1] :
                return "[Microcytic_Anemia] except thalassemia : \n - Anemia of Chronic Disease\n- chronic iron deficiency\n- hereditary spherocytosis\n- G6PD deficiency "
            # RDW high + MCV normal
            elif mcv[0]<= mcv <= mcv[1] and rdw > rdw[1] :
                return "[Normocytic_Anemia]: \n - Non-Hemolytic Anemia\n- Anemia of Chronic Disease\n- sickle cell anemia\n- G6PD deficiency "
            # RDW normal + MCV normal
            elif mcv[0]<= mcv <= mcv[1] and rdw < rdw[1] :
                return "[Normocytic_Anemia]: \n - myelodysplsia\n- blood loss\n- chronicillness\n- Hemolysis\n- chronic lymphocytic leukemia\n- chronic myeloid leukemia\n- hemoglobinopathy"
            # RDW high + MCV high
            elif mcv > mcv[1] and rdw > rdw[1] :
                return "[Macrocytic_Anemia]: \n - B12/Folat deficiency\n- Autoimmune hemolytic anemia"
            # RDW normal + MCV high 
            elif mcv > mcv[1] and rdw < rdw[1] :
                return "[Microcytic_Anemia]: \n- aplastic anemia\n- hepatopathy\n- preleukemia\n- myelodysplastic"
            else :
                return "Undefined Pattern"
        
        return "Undefined Pattern"

class AnemiaClassifier:
    """Handles detailed anemia classification based on MCV and RDW"""
    
    @staticmethod
    def calculate_mentzer_index(mcv: float, rbc: float) -> float:
        """Calculates Mentzer index (MCV/RBC)"""
        return mcv / rbc if rbc != 0 else 0
    
    @staticmethod
    def classify_anemia(mcv: float, rdw: float, rbc: float) -> Dict:
        """Classifies anemia type based on MCV and RDW values"""
        mentzer_index = AnemiaClassifier.calculate_mentzer_index(mcv, rbc)
        
        # RDW high + MCV low
        if rdw > rdw[1] and mcv < mcv[0]:
            if mentzer_index > 13:
                return {
                    "type": "Iron Deficiency Anemia",
                    "severity": Severity.WARNING,
                    "differential": [
                        "Chronic Illness",
                        "RBC Fragmentation",
                        "G6PD Deficiency"
                    ]
                }
            else:
                return {
                    "type": "Beta Thalassemia",
                    "severity": Severity.WARNING,
                    "differential": ["Hemoglobin H"]
                }

        # RDW normal + MCV low  
        if rdw < rdw[1] and mcv < mcv[0]:
            if mentzer_index < 13:
                return {
                    "type": "Thalassemia Trait",
                    "severity": Severity.WARNING,
                    "differential": ["Chronic Illness"]
                }
            else:
                return {
                    "type": "Late Iron Deficiency",
                    "severity": Severity.WARNING,
                    "differential": ["Hereditary Spherocytosis"]
                }

        # RDW high + MCV normal
        if rdw > rdw[1] and mcv[0] <= mcv <= mcv[1]:
            return {
                "type": "Early Mixed Deficiency",
                "severity": Severity.WARNING,
                "differential": [
                    "Early Iron Deficiency Anemia",
                    "Early B12/Folate Deficiency", 
                    "Sickle Cell Anemia"
                ]
            }

        # RDW normal + MCV normal
        if rdw < rdw[1] and mcv[0] <= mcv <= mcv[1]:
            return {
                "type": "Myelodysplasia",
                "severity": Severity.WARNING,
                "differential": [
                    "Blood Loss",
                    "Chronic Illness",
                    "Hemolysis",
                    "Chronic Lymphocytic Leukemia",
                    "Chronic Myeloid Leukemia",
                    "Hemoglobinopathy"
                ]
            }

        # RDW high + MCV high
        if rdw > rdw[1] and mcv > mcv[1]:
            return {
                "type": "B12/Folate Deficiency",
                "severity": Severity.WARNING,
                "differential": [
                    "Autoimmune Hemolytic Anemia",
                    "Cold Agglutinin",
                    "Alcoholism"
                ]
            }

        # RDW normal + MCV high
        if rdw < rdw[1] and mcv > mcv[1]:
            return {
                "type": "Aplastic Anemia",
                "severity": Severity.WARNING,
                "differential": [
                    "Hepatopathy",
                    "Preleukemia",
                    "Myelodysplastic",
                    "Alcoholism"
                ]
            }

        # RDW low
        if rdw < rdw[0]:
            return {
                "type": "Aplastic Anemia",
                "severity": Severity.WARNING,
                "differential": ["Thalassemia"]
            }
        
        return {"type": "Undefined Anemia", "severity": Severity.WARNING}

class WBCAnalyzer:
    """Handles WBC analysis and classification"""
    
    @staticmethod
    def analyze_wbc(wbc: float, neutrophils: float, lymphocytes: float) -> Dict:
        """Analyzes WBC count and differential"""
        if wbc < 4000:
            return {
                "condition": "Leukopenia",
                "severity": Severity.URGENT,
                "details": "Low WBC count requiring immediate evaluation"
            }
        elif wbc > 11000:
            if neutrophils > 7500:
                return {
                    "condition": "Neutrophilic Leukocytosis",
                    "severity": Severity.WARNING,
                    "details": "Suggests bacterial infection"
                }
            elif lymphocytes > 4000:
                return {
                    "condition": "Lymphocytic Leukocytosis",
                    "severity": Severity.WARNING,
                    "details": "Suggests viral infection"
                }
            elif wbc > 25000:
                return {
                    "condition": "Critical Leukocytosis",
                    "severity": Severity.CRITICAL,
                    "details": "Possible leukemia - requires immediate evaluation"
                }
        
        return {
            "condition": "Normal WBC",
            "severity": Severity.NORMAL,
            "details": "WBC count within normal range"
        }

class PLTAnalyzer:
    """Handles platelet analysis and classification"""
    
    @staticmethod
    def analyze_plt(plt: float, mpv: float) -> Dict:
        """Analyzes platelet count and MPV"""
        if plt < 20000:
            return {
                "condition": "Severe Thrombocytopenia",
                "severity": Severity.CRITICAL,
                "details": "Critical platelet count - immediate evaluation required"
            }
        elif plt < 150000:
            return {
                "condition": "Thrombocytopenia",
                "severity": Severity.URGENT,
                "details": "Low platelet count requiring evaluation"
            }
        elif plt > 450000:
            if mpv > 11:
                return {
                    "condition": "Thrombocytosis",
                    "severity": Severity.WARNING,
                    "details": "Primary thrombocytosis suspected"
                }
            else:
                return {
                    "condition": "Reactive Thrombocytosis",
                    "severity": Severity.WARNING,
                    "details": "Secondary thrombocytosis suspected"
                }
        
        return {
            "condition": "Normal PLT",
            "severity": Severity.NORMAL,
            "details": "Platelet count within normal range"
        }

class CBCAnalyzer:
    """Main class for comprehensive CBC analysis"""
    
    def __init__(self):
        self.rbc_pattern = RBCPattern()
        self.anemia_classifier = AnemiaClassifier()
        self.wbc_analyzer = WBCAnalyzer()
        self.plt_analyzer = PLTAnalyzer()
    
    def analyze_cbc(self, result: CBCResult) -> Dict:
        """Performs comprehensive CBC analysis"""
        
        # Individual analyses
        rbc_analysis = self.analyze_rbc(result)
        wbc_analysis = self.wbc_analyzer.analyze_wbc(
            result.wbc, result.neutrophils, result.lymphocytes
        )
        plt_analysis = self.plt_analyzer.analyze_plt(result.plt, result.mpv)
        
        # Check for critical patterns
        critical_patterns = self.check_critical_patterns(result)
        
        # Compile final result
        return {
            "rbc_analysis": rbc_analysis,
            "wbc_analysis": wbc_analysis,
            "plt_analysis": plt_analysis,
            "critical_patterns": critical_patterns,
            "overall_severity": self.determine_overall_severity(
                rbc_analysis, wbc_analysis, plt_analysis, critical_patterns
            )
        }
    
    def analyze_rbc(self, result: CBCResult) -> Dict:
        """Analyzes RBC parameters"""
        # Convert numeric values to categorical (Low/Normal/High)
        hgb_cat = self.categorize_hgb(result.hgb)
        rbc_cat = self.categorize_rbc(result.rbc)
        hct_cat = self.categorize_hct(result.hct)
        
        pattern = self.rbc_pattern.classify_pattern(hgb_cat, rbc_cat, hct_cat)
        
        if pattern == "Anemia":
            return self.anemia_classifier.classify_anemia(
                result.mcv, result.rdw, result.rbc
            )
        
        return {
            "pattern": pattern,
            "severity": Severity.NORMAL if pattern == "Normal Pattern" else Severity.WARNING
        }
    
    def check_critical_patterns(self, result: CBCResult) -> List[Dict]:
        """Checks for critical patterns combining multiple parameters"""
        critical_patterns = []
        
        # Check for pancytopenia
        if (result.hgb < 10 and result.wbc < 4000 and result.plt < 150000):
            critical_patterns.append({
                "condition": "Pancytopenia",
                "severity": Severity.CRITICAL,
                "details": "All cell lines low - bone marrow evaluation needed"
            })
        
        # Check for leukemia pattern
        if (result.wbc > 25000 and result.hgb < 10 and 
            (result.plt < 50000 or result.plt > 1000000)):
            critical_patterns.append({
                "condition": "Acute Leukemia Pattern",
                "severity": Severity.CRITICAL,
                "details": "Immediate hematology consultation needed"
            })
        
        return critical_patterns
    
    @staticmethod
    def determine_overall_severity(
        rbc: Dict, wbc: Dict, plt: Dict, critical: List[Dict]
    ) -> Severity:
        """Determines overall severity based on all analyses"""
        if any(pattern["severity"] == Severity.CRITICAL for pattern in critical):
            return Severity.CRITICAL
        
        severities = [
            rbc.get("severity", Severity.NORMAL),
            wbc.get("severity", Severity.NORMAL),
            plt.get("severity", Severity.NORMAL)
        ]
        
        if Severity.CRITICAL in severities:
            return Severity.CRITICAL
        elif Severity.URGENT in severities:
            return Severity.URGENT
        elif Severity.WARNING in severities:
            return Severity.WARNING
        return Severity.NORMAL
   
    # تعديل دوال التصنيف فقط
    def categorize_hgb(self, value: float, patient: Patient) -> str:
        """تصنيف قيمة الهيموجلوبين باستخدام القيم المرجعية"""
        try:
            range = CBCReferenceValues.get_reference_range(
                test_type="hgb",
                age_months=patient.age_months,
                gender=patient.gender,
                is_pregnant=patient.is_pregnant,
                trimester=patient.pregnancy_trimester
            )
            return range.categorize(value)
        except ValueError:
            # في حالة عدم توفر القيم المرجعية، استخدم التصنيف القديم
            if value < 12: return "Low"
            if value > 16: return "High"
            return "Normal"

    def categorize_rbc(self, value: float, patient: Patient) -> str:
        """تصنيف قيمة كرات الدم الحمراء باستخدام القيم المرجعية"""
        try:
            range = CBCReferenceValues.get_reference_range(
                test_type="rbc",
                age_months=patient.age_months,
                gender=patient.gender,
                is_pregnant=patient.is_pregnant,
                trimester=patient.pregnancy_trimester
            )
            return range.categorize(value)
        except ValueError:
            # في حالة عدم توفر القيم المرجعية، استخدم التصنيف القديم
            if value < 4: return "Low"
            if value > 6: return "High"
            return "Normal"

    def categorize_hct(self, value: float, patient: Patient) -> str:
        """تصنيف قيمة الهيماتوكريت باستخدام القيم المرجعية"""
        try:
            range = CBCReferenceValues.get_reference_range(
                test_type="hct",
                age_months=patient.age_months,
                gender=patient.gender,
                is_pregnant=patient.is_pregnant,
                trimester=patient.pregnancy_trimester
            )
            return range.categorize(value)
        except ValueError:
            # في حالة عدم توفر القيم المرجعية، استخدم التصنيف القديم
            if value < 36: return "Low"
            if value > 48: return "High"
            return "Normal"

    def analyze_rbc(self, result: CBCResult, patient: Patient) -> Dict:
        """تحليل معايير RBC"""
        # تحويل القيم الرقمية إلى تصنيفات (منخفض/طبيعي/مرتفع)
        hgb_cat = self.categorize_hgb(result.hgb, patient)
        rbc_cat = self.categorize_rbc(result.rbc, patient)
        hct_cat = self.categorize_hct(result.hct, patient)
        
        pattern = self.rbc_pattern.classify_pattern(hgb_cat, rbc_cat, hct_cat)
        
        if pattern == "Anemia":
            return self.anemia_classifier.classify_anemia(
                result.mcv, result.rdw, result.rbc
            )
        
        return {
            "pattern": pattern,
            "severity": Severity.NORMAL if pattern == "Normal Pattern" else Severity.WARNING
        }
    
#     # Helper methods for categorization
#     @staticmethod
#     def categorize_hgb(value: float) -> str:
#         if value < 12: return "Low"
#         if value > 16: return "High"
#         return "Normal"
    
#     @staticmethod
#     def categorize_rbc(value: float) -> str:
#         if value < 4: return "Low"
#         if value > 6: return "High"
#         return "Normal"
    
#     @staticmethod
#     def categorize_hct(value: float) -> str:
#         if value < 36: return "Low"
#         if value > 48: return "High"
#         return "Normal"

# Example usage
def main():
    # Create sample CBC result
    sample_result = CBCResult(
        hgb=12,    # Low
        rbc=5,    # Low
        hct=32,     # Low
        mcv=75,     # Low
        rdw=16.5,   # High
        wbc=13000,  # High
        neutrophils=4000,  # High
        lymphocytes=2000,   # Normal
        plt=140000, # Low
        mpv=10.5    # Normal
    )
    
    # Create analyzer and analyze sample
    analyzer = CBCAnalyzer()
    result = analyzer.analyze_cbc(sample_result)
    
    # Print results
    print("CBC Analysis Results:")
    print("--------------------")
    print(f"RBC Analysis: {result['rbc_analysis']}")
    print(f"WBC Analysis: {result['wbc_analysis']}")
    print(f"PLT Analysis: {result['plt_analysis']}")
    print("\nCritical Patterns:")
    for pattern in result['critical_patterns']:
        print(f"- {pattern['condition']}: {pattern['details']}")
    print(f"\nOverall Severity: {result['overall_severity'].value}")

if __name__ == "__main__":
    main()

CBC Analysis Results:
--------------------
RBC Analysis: {'pattern': 'Normal Pattern', 'severity': <Severity.NORMAL: 'Normal'>}
WBC Analysis: {'condition': 'Normal WBC', 'severity': <Severity.NORMAL: 'Normal'>, 'details': 'WBC count within normal range'}
PLT Analysis: {'condition': 'Thrombocytopenia', 'severity': <Severity.URGENT: 'Urgent'>, 'details': 'Low platelet count requiring evaluation'}

Critical Patterns:

Overall Severity: Urgent


In [17]:
import pandas as pd
from typing import Dict, List, Tuple
from dataclasses import dataclass
from enum import Enum
import json

class CBCPattern:
    """Class to identify and analyze CBC patterns"""
    def __init__(self, rbc: float, hgb: float, hct: float, mcv: float, rdw: float):
        self.rbc = rbc
        self.hgb = hgb
        self.hct = hct
        self.mcv = mcv
        self.rdw = rdw
        self.ref_ranges = self._get_reference_ranges()
        
    def _get_pattern_type(self) -> str:
        """Get RBC-HGB-HCT pattern type"""
        rbc_state = self._get_state(self.rbc, self.ref_ranges["RBC"])
        hgb_state = self._get_state(self.hgb, self.ref_ranges["HGB"])
        hct_state = self._get_state(self.hct, self.ref_ranges["HCT"])
        return f"{hgb_state}_{rbc_state}_{hct_state}"

    def _get_state(self, value: float, range_tuple: Tuple[float, float]) -> str:
        if value < range_tuple[0]:
            return "Low"
        elif value > range_tuple[1]:
            return "High"
        return "Normal"

    def analyze(self) -> Dict:
        """Main analysis method"""
        pattern = self._get_pattern_type()
        
        # Wrong measurements patterns
        wrong_patterns = [
            "Low_Low_High", "Low_Normal_High", "Low_High_High",
            "Normal_Low_High", "Normal_High_Low", "High_Low_Low",
            "High_Low_High", "High_Low_Normal", "High_Normal_Low",
            "High_High_Low", "High_High_Normal"
        ]
        
        if pattern in wrong_patterns:
            return {
                "category": "Wrong_Measurement",
                "description": "Laboratory error suspected",
                "recommendation": "Repeat CBC test"
            }

        # Process normal patterns
        if pattern in ["Normal_Normal_Low", "Normal_Normal_Normal", "Normal_Normal_High"]:
            return self._analyze_normal_pattern(pattern)
            
        # Process early microcytosis patterns
        if pattern in ["Low_High_Low", "Low_High_Normal"]:
            return {
                "category": "Early_Microcytosis",
                "description": "Compensatory response detected",
                "details": "High RBC production to compensate for low HGB",
                "possible_causes": ["Early iron deficiency", "Early thalassemia"]
            }

        # Process anemia patterns
        if pattern in ["Low_Low_Low", "Low_Low_Normal", "Low_Normal_Low", "Low_Normal_Normal"]:
            return self._analyze_anemia_pattern()

        return {"category": "Unknown_Pattern", "description": "Pattern not recognized"}

    def _analyze_normal_pattern(self, pattern: str) -> Dict:
        """Analyze normal range patterns"""
        result = {"category": "Normal_Pattern"}
        
        if pattern in ["Normal_Normal_Low", "Normal_Normal_Normal"]:
            if self.mcv < 75:
                result.update({
                    "subcategory": "Early_Microcytic",
                    "possible_causes": ["Early iron deficiency", "Beta thalassemia trait"]
                })
            elif 75 <= self.mcv <= 100:
                result.update({"subcategory": "Normal_CBC"})
                
        elif pattern == "Normal_Normal_High":
            if self.mcv > 100:
                result.update({
                    "subcategory": "Early_Macrocytic",
                    "possible_causes": ["B12 deficiency", "Folate deficiency"]
                })
            elif self.mcv <= 100:
                result.update({
                    "subcategory": "Probable_Dehydration"
                })
                
        return result

    def _analyze_anemia_pattern(self) -> Dict:
        """Analyze anemia patterns based on MCV and RDW"""
        result = {"category": "Anemia"}
        
        # Calculate Mentzer index
        mentzer_index = self.mcv / self.rbc
        
        # RDW high MCV Low
        if self.rdw > 14.5 and self.mcv < 80:
            result.update({
                "subcategory": "RDW_High_MCV_Low",
                "possible_causes": [
                    "Iron deficiency anemia" if mentzer_index > 13 else "Beta thalassemia",
                    "Chronic illness",
                    "RBC fragmentation",
                    "Hemoglobin H",
                    "G6PD deficiency"
                ],
                "notes": f"Mentzer index: {mentzer_index:.1f}"
            })
            
        # RDW normal MCV low
        elif self.rdw <= 14.5 and self.mcv < 80:
            result.update({
                "subcategory": "RDW_Normal_MCV_Low",
                "possible_causes": [
                    "Thalassemia trait" if mentzer_index < 13 else "Late iron deficiency",
                    "Hereditary spherocytosis",
                    "Chronic illness"
                ],
                "notes": f"Mentzer index: {mentzer_index:.1f}"
            })
            
        # RDW high MCV normal
        elif self.rdw > 14.5 and 80 <= self.mcv <= 100:
            result.update({
                "subcategory": "RDW_High_MCV_Normal",
                "possible_causes": [
                    "Early iron deficiency anemia",
                    "Early B12/Folate deficiency",
                    "Sickle cell anemia"
                ]
            })
            
        # RDW normal MCV normal
        elif self.rdw <= 14.5 and 80 <= self.mcv <= 100:
            result.update({
                "subcategory": "RDW_Normal_MCV_Normal",
                "possible_causes": [
                    "Myelodysplasia",
                    "Blood loss",
                    "Chronic illness",
                    "Hemolysis",
                    "CLL/CML",
                    "Hemoglobinopathy"
                ]
            })
            
        # RDW high MCV high
        elif self.rdw > 14.5 and self.mcv > 100:
            result.update({
                "subcategory": "RDW_High_MCV_High",
                "possible_causes": [
                    "B12/Folate deficiency",
                    "Autoimmune hemolytic anemia",
                    "Cold agglutinin",
                    "Alcoholism"
                ]
            })
            
        # RDW normal MCV high
        elif self.rdw <= 14.5 and self.mcv > 100:
            result.update({
                "subcategory": "RDW_Normal_MCV_High",
                "possible_causes": [
                    "Aplastic anemia",
                    "Hepatopathy",
                    "Preleukemia",
                    "Myelodysplastic",
                    "Alcoholism"
                ]
            })
            
        # RDW low (rare cases)
        elif self.rdw < 11.5:
            result.update({
                "subcategory": "RDW_Low",
                "possible_causes": [
                    "Aplastic anemia",
                    "Thalassemia"
                ]
            })
            
        return result

    def _get_reference_ranges(self) -> Dict:
        """Get reference ranges"""
        return {
            "RBC": (4.5, 5.9),
            "HGB": (13.5, 17.5),
            "HCT": (41.0, 53.0),
            "MCV": (80.0, 100.0),
            "RDW": (11.5, 14.5)
        }

class CBCAnalyzer:
    def __init__(self, data: Dict):
        self.data = data
        self.results = {}
        
    def analyze(self) -> Dict:
        """Perform complete CBC analysis"""
        # Analyze RBC pattern
        rbc_pattern = CBCPattern(
            self.data["RBC"],
            self.data["HGB"],
            self.data["HCT"],
            self.data["MCV"],
            self.data["RDW"]
        )
        self.results["RBC_ANALYSIS"] = rbc_pattern.analyze()
        
        # Add WBC and PLT analysis here...
        
        return self.results

# Example usage
if __name__ == "__main__":
    sample_data = {
        "RBC": 4.8,
        "HGB": 14.0,
        "HCT": 42.0,
        "MCV": 88.0,
        "RDW": 13.0,
        # Add other parameters...
    }
    
    
    analyzer = CBCAnalyzer(sample_data)
    results = analyzer.analyze()
    print(json.dumps(results, indent=2))

{
  "RBC_ANALYSIS": {
    "category": "Normal_Pattern",
    "subcategory": "Normal_CBC"
  }
}
